In [17]:
import subprocess
import csv
import os 
import sys
from tqdm import tqdm
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_dir)
from .utils import get_video_fps

# Đường dẫn đến tệp CSV chứa thông tin các cảnh
csv_file = 'scene_durations_v1.csv'

# Đường dẫn đến video gốc
input_video_files = "../data/sample_tv360/video_chunk/chunk_1.mp4" # Điều chỉnh đường dẫn video của bạn

# Đường dẫn đến video đầu ra
output_video_file = 'output_video.mp4'

# Đọc thông tin từ tệp CSV
scenes = []
with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        scenes.append({
            'scene': int(row['Scene']),
            'start_time': row['Start Time'],
            'end_time': row['End Time'],
            'duration': float(row['Duration (seconds)'])
        })

# Lọc và nhóm các cảnh liên tục
grouped_scenes = []
current_group = []

current_scene = []
for i, scene in enumerate(scenes):
    duration = scene['duration']
    if duration <10 :
        current_scene.append(scene)
    elif duration <30 and i+1 < len(scenes) and scenes[i+1]['duration'] < 10:
        current_scene.append(scene)
    elif i== len(scenes):
        current_scene.append(scene)
    else:
        grouped_scenes.append(current_scene)
        current_scene = []

grouped_scenes = sorted([i for i in grouped_scenes if i != []], 
                        key= lambda group: (sum(scene['duration'] for scene in group), len(group)), reverse=True)
chosen_scenes = grouped_scenes[0]


# Lấy các scene liên tục
def get_consecutive_scenes(grouped_scenes, taken_count =4):
    selected_scenes = []  # List to store the final scenes

    for group in grouped_scenes:
        if len(group) > taken_count:
            # If the group has more than 5 scenes, take every 5 consecutive scenes
            for i in range(0, len(group), taken_count):
                selected_scenes.append(group[i:i+taken_count])
        else:
            # If the group has 5 or fewer scenes, take the entire group
            pass

    return selected_scenes
selected_scenes = get_consecutive_scenes(grouped_scenes, taken_count =4)

In [29]:
fps = get_video_fps('../sample_tv360.mp4')
ffmpeg_commands = []
chunk = input_video_files.split('/')[-1].split('_')[-1].split('.')[0]

for group in tqdm(selected_scenes):
#     i = 1
    # output_video_file = 'output_video_for_chunk_{chunk}_part_{i}'
    for scene in tqdm(group):
        start_time = scene['start_time']
        end_time = scene['end_time']
        duration = scene['duration']
            # Sao chép codec cho các đoạn dài hơn
        cmd = [
                'ffmpeg',
                '-y',
                '-i', input_video_files,
                '-ss', start_time,
                '-to', end_time,
                '-c:v', 'libopenh264',
                '-c:a', 'copy',
                f'temp_scene_{scene["scene"]}_chunk_{chunk}.mp4'
            ]
        ffmpeg_commands.append(cmd)
for cmd in tqdm(ffmpeg_commands):
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Sort based on the scene['scene'] value
temp_files = sorted(
    [f"temp_scene_{scene['scene']}_chunk_{chunk}.mp4" for group in selected_scenes for scene in group],
    key=lambda x: int(x.split('_')[2])  # Extracts the 'scene' part from the filename
)

with open(f'file_list_at_chunk_{chunk}.txt', 'w') as f:
    for temp_file in temp_files:
        f.write(f"file '{temp_file}'\n")


concat_cmd = [
    'ffmpeg',
    '-y',
    '-f', 'concat',
    '-safe', '0',
    '-i', f'file_list_at_chunk_{chunk}.txt',
    '-c:v', 'copy',
    '-c:a', 'copy',
    '-r', str(fps),
    output_video_file
]

# Chạy lệnh FFmpeg để nối video
subprocess.run(concat_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

print(f"Video liên tục đã được tạo thành công: {output_video_file}")

100%|██████████| 37/37 [08:19<00:00, 13.49s/it]

Video liên tục đã được tạo thành công: output_video.mp4


In [66]:
concat_cmd = [
    'ffmpeg',
    '-y',
    '-f', 'concat',
    '-safe', '0',
    '-i', f'file_list_at_chunk_{chunk}.txt',
    '-c:v', 'copy',
    '-c:a', 'copy',
    '-r', str(fps),
    output_video_file
]

# Chạy lệnh FFmpeg để nối video
subprocess.run(concat_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

print(f"Video liên tục đã được tạo thành công: {output_video_file}")

Video liên tục đã được tạo thành công: output_video.mp4


In [69]:
import os
import glob

# Đường dẫn đến thư mục chứa video
folder_path = os.getcwd()  # Thay bằng đường dẫn thực tế

# Tìm tất cả các file có dạng 'temp_scene*.mp4' trong folder
temp_files = glob.glob(os.path.join(folder_path, 'temp_scene*.mp4'))

# Xóa từng file trong danh sách
for temp_file in temp_files:
    try:
        os.remove(temp_file)
        print(f"Đã xóa: {temp_file}")
    except Exception as e:
        print(f"Lỗi khi xóa {temp_file}: {e}")


Đã xóa: /home/thiendc/projects/video_summarization/scene_based/temp_scene_4_chunk_1.mp4
Đã xóa: /home/thiendc/projects/video_summarization/scene_based/temp_scene_45_chunk_1.mp4
Đã xóa: /home/thiendc/projects/video_summarization/scene_based/temp_scene_31_chunk_1.mp4
Đã xóa: /home/thiendc/projects/video_summarization/scene_based/temp_scene_33_chunk_1.mp4
Đã xóa: /home/thiendc/projects/video_summarization/scene_based/temp_scene_39_chunk_1.mp4
Đã xóa: /home/thiendc/projects/video_summarization/scene_based/temp_scene_5_chunk_1.mp4
Đã xóa: /home/thiendc/projects/video_summarization/scene_based/temp_scene_30_chunk_1.mp4
Đã xóa: /home/thiendc/projects/video_summarization/scene_based/temp_scene_34_chunk_1.mp4
Đã xóa: /home/thiendc/projects/video_summarization/scene_based/temp_scene_19_chunk_1.mp4
Đã xóa: /home/thiendc/projects/video_summarization/scene_based/temp_scene_24_chunk_1.mp4
Đã xóa: /home/thiendc/projects/video_summarization/scene_based/temp_scene_29_chunk_1.mp4
Đã xóa: /home/thiendc/p